Initial Testing of a Decoder-only (GPT style) architecture.

# NOTE:

The training of the tokenizer and other items was done on the FULL dataset.  This causes a data leak and proper train/val/test splitting should be done in the final product.

# Imports

In [5]:
import os

import tokenizers
from tokenizers import Tokenizer, models, pre_tokenizers, trainers

import transformers
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset, load_dataset #, load_metric

import tensorflow as tf


import ast # Because the tokenized_text column data is stored as a string instead of a list...
import pandas as pd


# Grab the dataset

1) Get the dataset
2) Fix the tokenize_text column from a string to a list of strings
3) Pull out all of the words into a mega-list

In [18]:
# Read the processed card data from wherever you have it.

# df = pd.read_csv('../data/processed/mtg_carddata_processed.csv')
df = pd.read_csv('..\data\processed\mtg_carddata_processed_2_23_25.csv')

In [19]:
df

,name,mana_cost,type_line,oracle_text,power,toughness,colors,keywords,mtgo_id,loyalty,defense,processed_oracle_text,tokenized_text,tfidf_vector
0,"Nissa, Worldsoul Speaker",{3}{G},Legendary Creature — Elf Druid,"Landfall — Whenever a land you control enters,...",3,3,['G'],['Landfall'],NaN,NaN,NaN,Landfall — Whenever a land you control enters ...,"['Landfall', 'Whenever', 'a', 'land', 'you', '...",[0. 0. 0. ... 0. 0. 0.]
1,Static Orb,{3},Artifact,"As long as <name> is untapped, players can't u...",NaN,NaN,[],[],15870.0,NaN,NaN,"As long as Static Orb is untapped , players ca...","['As', 'long', 'as', '<name>', 'is', 'untapped...",[0. 0. 0. ... 0. 0. 0.]
2,Sensory Deprivation,{U},Enchantment — Aura,Enchant creature\r\nEnchanted creature gets -3...,NaN,NaN,['U'],['Enchant'],49283.0,NaN,NaN,Enchant creature \n Enchanted creature gets -3...,"['Enchant', 'creature', '\\n', 'Enchanted', 'c...",[0. 0. 0. ... 0. 0. 0.]
3,Road of Return,{G}{G},Sorcery,Choose one —\r\n• Return target permanent card...,NaN,NaN,['G'],['Entwine'],77122.0,NaN,NaN,Choose one — \n • Return target permanent card...,"['Choose', 'one', '\\n', 'Return', 'target', '...",[0. 0. 0. ... 0. 0. 0.]
4,Storm Crow,{1}{U},Creature — Bird,Flying (This creature can't be blocked except ...,1,2,['U'],['Flying'],22609.0,NaN,NaN,Flying (This creature can't be blocked except ...,"['Flying', 'This', 'creature', ""can't"", 'be', ...",[0. 0. 0. ... 0. 0. 0.]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30618,Devoted Hero,{W},Creature — Elf Soldier,NaN,1,2,['W'],[],NaN,NaN,NaN,NaN,[],[0. 0. 0. ... 0. 0. 0.]
30619,Without Weakness,{1}{B},Instant,Target creature you control gains indestructib...,NaN,NaN,['B'],['Cycling'],64646.0,NaN,NaN,Target creature you control gains indestructib...,"['Target', 'creature', 'you', 'control', 'gain...",[0. 0. 0. ... 0. 0. 0.]
30620,Firesong and Sunspeaker,{4}{R}{W},Legendary Creature — Minotaur Cleric,Red instant and sorcery spells you control hav...,4,6,"['R', 'W']",[],101914.0,NaN,NaN,Red instant and sorcery spells you control hav...,"['Red', 'instant', 'and', 'sorcery', 'spells',...",[0. 0. 0. ... 0. 0. 0.]
30621,"Samut, the Tested",{2}{R}{G},Legendary Planeswalker — Samut,+1: Up to one target creature gains double str...,NaN,NaN,"['G', 'R']",[],64772.0,4,NaN,+1 : Up to one target creature gains double st...,"['1', ':', 'Up', 'to', 'one', 'target', 'creat...",[0. 0. 0. ... 0. 0. 0.]


In [20]:
df['tokenized_text'][0]

"['Landfall', 'Whenever', 'a', 'land', 'you', 'control', 'enters', ',', 'you', 'get', '{E}', '{E}', 'two', 'energy', 'counters', '.', '\\\\n', 'You', 'may', 'pay', 'eight', '{E}', 'rather', 'than', 'pay', 'the', 'mana', 'cost', 'for', 'permanent', 'spells', 'you', 'cast', '.']"

In [21]:
import ast

# Apply ast.literal_eval to each row in the tokenized_text column
df['tokenized_text'] = df['tokenized_text'].apply(ast.literal_eval)

# Display first few rows to verify
df['tokenized_text'].head()

0    [Landfall, Whenever, a, land, you, control, en...
1    [As, long, as, <name>, is, untapped, ,, player...
2    [Enchant, creature, \n, Enchanted, creature, g...
3    [Choose, one, \n, Return, target, permanent, c...
4    [Flying, This, creature, can't, be, blocked, e...
Name: tokenized_text, dtype: object

In [22]:
df['tokenized_text'][0][0]

'Landfall'

In [23]:
# Combine all tokens into one large list
all_tokens = []
for tokens in df['tokenized_text']:
    all_tokens.extend(tokens)

# Alternative one-liner using list comprehension
# all_tokens = [token for tokens in df['tokenized_text'] for token in tokens]

# Display the first 20 tokens to verify
print(f"Total tokens: {len(all_tokens)}")
print("First 20 tokens:", all_tokens[:20])

Total tokens: 968000
First 20 tokens: ['Landfall', 'Whenever', 'a', 'land', 'you', 'control', 'enters', ',', 'you', 'get', '{E}', '{E}', 'two', 'energy', 'counters', '.', '\\n', 'You', 'may', 'pay']


In [24]:
corpus = df['processed_oracle_text']
corpus

0        Landfall — Whenever a land you control enters ...
1        As long as Static Orb is untapped , players ca...
2        Enchant creature \n Enchanted creature gets -3...
3        Choose one — \n • Return target permanent card...
4        Flying (This creature can't be blocked except ...
                               ...                        
30618                                                  NaN
30619    Target creature you control gains indestructib...
30620    Red instant and sorcery spells you control hav...
30621    +1 : Up to one target creature gains double st...
30622                     All Sliver creatures get +1/+1 .
Name: processed_oracle_text, Length: 30623, dtype: object

In [25]:
# Convert corpus to list of strings if it's not already
corpus_list = corpus.tolist() if hasattr(corpus, 'tolist') else list(corpus)


In [26]:
type(corpus)

pandas.core.series.Series

In [27]:
type(corpus_list[0])

str

In [28]:
print(corpus[0:3])

0    Landfall — Whenever a land you control enters ...
1    As long as Static Orb is untapped , players ca...
2    Enchant creature \n Enchanted creature gets -3...
Name: processed_oracle_text, dtype: object


In [29]:
# Convert pandas Series to list of strings
corpus_list = corpus.values.tolist()

# Ensure all elements are strings
corpus_list = [str(text) for text in corpus_list]



In [30]:
corpus_list[0]

'Landfall — Whenever a land you control enters , you get {E} {E} (two energy counters) . \\n You may pay eight {E} rather than pay the mana cost for permanent spells you cast .'

In [31]:
print(type(corpus_list))  # Should show: <class 'list'>
print(type(corpus_list[0]))  # Should show: <class 'str'>

<class 'list'>
<class 'str'>


# Tokenize

In [33]:
# Create a directory for tokenizer files if it doesn't exist
models_dir = "../models"
os.makedirs(models_dir, exist_ok=True)

# Initialize trainer with specific output directory
tk_trainer = tokenizers.trainers.BpeTrainer(
    vocab_size=8192,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
)

# Initialize tokenizer with output directory
tokenizer = tokenizers.Tokenizer(tokenizers.models.BPE())
tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.Whitespace()

tokenizer.enable_padding(pad_id=0, pad_token="[PAD]")
tokenizer.enable_truncation(max_length=512) # Maybe we cut this down to 258?

# Train the tokenizer
tokenizer.train_from_iterator(corpus_list, trainer=tk_trainer)

# Save the tokenizer
tokenizer.save(os.path.join(models_dir, "tokenizer_leak.json")) # Your tokenizer may have been trained with the full dataset causing a data leak...

In [34]:
# Verify the tokenizer works
sample_text = corpus_list[0]
encoded = tokenizer.encode(sample_text)
print(f"Encoded: {encoded.tokens}")

Encoded: ['Landfall', '—', 'Whenever', 'a', 'land', 'you', 'control', 'enters', ',', 'you', 'get', '{', 'E', '}', '{', 'E', '}', '(', 'two', 'energy', 'counters', ')', '.', '\\', 'n', 'You', 'may', 'pay', 'eight', '{', 'E', '}', 'rather', 'than', 'pay', 'the', 'mana', 'cost', 'for', 'permanent', 'spells', 'you', 'cast', '.']


In [35]:
# Test the tokenizer on a string with a tab character and non-sensical text
test = tokenizer.encode("This is a test of \\t nonzensicallicalness {U}")
print(test.tokens)

['This', 'is', 'a', 'test', 'of', '\\', 't', 'non', 'z', 'ens', 'ic', 'all', 'ical', 'ness', '{', 'U', '}']


## Load your Tokenizer

In [ ]:
# Since you have a pre-trained tokenizer, you can now load it directly
tokenizer = Tokenizer.from_file(os.path.join(models_dir, "tokenizer_leak.json"))

# Build the Decoder

In [27]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embed_dims, seq_len):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size + 1, embed_dims)
        self.pos_embedding = tf.keras.layers.Embedding(seq_len, embed_dims, mask_zero=True)
        
    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)
    
    def call(self, x):
        index_range = tf.shape(x)[-1]
        y = self.embedding(x)
        indices = tf.range(index_range)
        pos = self.pos_embedding(indices)
        return y + pos

In [28]:
class DecoderSelfAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim):
        super().__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=key_dim)
        
    def call(self, x):
        y = self.attention(
            query=x,
            value=x,
            key=x,
            use_causal_mask=True)
        return y

In [29]:
VOCAB_SIZE = 8912 # Define this based on training data vocab size.
SEQUENCE_LENGTH = 500 # Define this based on training data sequence length.
EMBEDDING_DIM = 512 # Configure
FEED_FORWARD_DIM = 4 * EMBEDDING_DIM # This is usual value used by other LLMs, from below reference.
DROPOUT = 0.5 # Configure
NUM_HEADS = 16 # Configure
NUM_DECODER_LAYERS = 4 # Configure

### Reference: https://cameronrwolfe.substack.com/p/decoder-only-transformers-the-workhorse

decoder_inputs = tf.keras.Input(shape=(None,), dtype=tf.int64, name="decoder_inputs")
positional_embedding = PositionalEmbedding(
    VOCAB_SIZE, EMBEDDING_DIM, SEQUENCE_LENGTH)(decoder_inputs)

layer_input = positional_embedding
layer_output = None
for i in range(NUM_DECODER_LAYERS):
    decoder_norm_1 = tf.keras.layers.LayerNormalization()(layer_input)
    decoder_self_attention = DecoderSelfAttention(NUM_HEADS, EMBEDDING_DIM)(decoder_norm_1)
    decoder_add_1 = tf.keras.layers.Add()([decoder_self_attention, decoder_norm_1])
    decoder_norm_2 = tf.keras.layers.LayerNormalization()(decoder_add_1)
    decoder_feedforward_1 = tf.keras.layers.Dense(FEED_FORWARD_DIM, activation="gelu")(decoder_norm_2)
    decoder_feedforward_2 = tf.keras.layers.Dense(EMBEDDING_DIM)(decoder_feedforward_1)
    decoder_dropout = tf.keras.layers.Dropout(DROPOUT)(decoder_feedforward_2)
    decoder_add_2 = tf.keras.layers.Add()([decoder_dropout, decoder_add_1])

    layer_input = decoder_add_2
    layer_output = decoder_add_2

prediction = tf.keras.layers.Dense(VOCAB_SIZE, activation="softmax")(layer_output)
transformer = tf.keras.models.Model(
    inputs=decoder_inputs, outputs=prediction, name="transformer")

In [30]:
transformer.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 512) │  4,819,456 │ decoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, None, 512) │      1,024 │ positional_embed… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_self_atten… │ (None, None, 512) │ 16,802,304 │ layer_normalizat… │
│ (DecoderSelfAttent… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_76 (Add)        │ (None, None, 512) │          0 │ decoder_self_att… │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, None, 512) │      1,024 │ add_76[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_81 (Dense)    │ (None, None,      │  1,050,624 │ layer_normalizat… │
│                     │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_82 (Dense)    │ (None, None, 512) │  1,049,088 │ dense_81[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_76          │ (None, None, 512) │          0 │ dense_82[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_77 (Add)        │ (None, None, 512) │          0 │ dropout_76[0][0], │
│                     │                   │            │ add_76[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, None, 512) │      1,024 │ add_77[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_self_atten… │ (None, None, 512) │ 16,802,304 │ layer_normalizat… │
│ (DecoderSelfAttent… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_78 (Add)        │ (None, None, 512) │          0 │ decoder_self_att… │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, None, 512) │      1,024 │ add_78[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_83 (Dense)    │ (None, None,      │  1,050,624 │ layer_normalizat… │
│                     │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_84 (Dense)    │ (None, None, 512) │  1,049,088 │ dense_83[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_78          │ (None, None, 512) │          0 │ dense_84[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 85,007,568 (324.28 MB)

 Trainable params: 85,007,568 (324.28 MB)

 Non-trainable params: 0 (0.00 B)

# Tokenize your train/val/test

## Set up datasets for loading data

# Training Loop

# Setup Training Arguments

OLD CODING WORK:

# Set up the Trainer

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, return_tensors='pt')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_reduced['train'],
    eval_dataset=tokenized_dataset_reduced['validation'],0
    tokenizer=tokenizer,
    data_collator=data_collator
)
